In [1]:
import pymorphy2
import nltk
import re
#import wget

In [52]:
from ufal.udpipe import Model, Pipeline

In [2]:
morph = pymorphy2.MorphAnalyzer()


In [4]:
huis = ["три", "великой", "зачем", "маму"]
for hui in huis:
    print(morph.parse(hui))

[Parse(word='три', tag=OpencorporaTag('NUMR inan,accs'), normal_form='три', score=0.571428, methods_stack=((<DictionaryAnalyzer>, 'три', 2907, 4),)), Parse(word='три', tag=OpencorporaTag('NUMR nomn'), normal_form='три', score=0.285714, methods_stack=((<DictionaryAnalyzer>, 'три', 2907, 0),)), Parse(word='три', tag=OpencorporaTag('VERB,impf,tran sing,impr,excl'), normal_form='тереть', score=0.142857, methods_stack=((<DictionaryAnalyzer>, 'три', 2339, 11),))]
[Parse(word='великой', tag=OpencorporaTag('ADJF femn,sing,gent'), normal_form='великий', score=0.655172, methods_stack=((<DictionaryAnalyzer>, 'великой', 668, 8),)), Parse(word='великой', tag=OpencorporaTag('ADJF femn,sing,ablt'), normal_form='великий', score=0.137931, methods_stack=((<DictionaryAnalyzer>, 'великой', 668, 11),)), Parse(word='великой', tag=OpencorporaTag('ADJF femn,sing,datv'), normal_form='великий', score=0.103448, methods_stack=((<DictionaryAnalyzer>, 'великой', 668, 9),)), Parse(word='великой', tag=OpencorporaTag(

In [31]:
with open('test.txt', 'r', encoding='utf-8') as f:
    text = f.read()

tokens = nltk.word_tokenize(text)
#print(tokens)

def goldenit(m):
    tags_cl = str(m.tag)
    if ' ' in tags_cl or ',' in tags_cl:
        ts = re.sub(' ', ',', tags_cl)
        tags = ts.split(',')
        mPOS = tags[0]
        nPOS = ','.join(tags[1:])
    else:
        nPOS = ''
        mPOS = str(m.tag)
    gold = (m.word, m.normal_form, mPOS, nPOS)
    return gold

golden_st = []

for token in tokens:
    m = morph.parse(token)[0]
    if m.score >= 0.95:
        gold = goldenit(m)
        golden_st.append(gold)
    else:
        for item in morph.parse(token):
            gold = goldenit(item)
            golden_st.append(gold)
    

def wrtitdown(fname, ready_tokens):
    with open(fname, 'w', encoding='utf-8') as f:
        f.write('Wordform_GS2\tLemma_GS2\tPOS_GS2\tGram_GS2\n')
        for token in ready_tokens:
            mes = '\t'.join(token)
            mes += '\n'
            f.write(mes)
            
wrtitdown('pymorph_standard.txt', golden_st)

In [54]:
model_url = 'https://rusvectores.org/static/models/udpipe_syntagrus.model'
modelfile = wget.download(model_url)
model = Model.load(modelfile)
process_pipeline = Pipeline(model, 'tokenize', Pipeline.DEFAULT, Pipeline.DEFAULT, 'conllu')

100% [........................................................................] 40616122 / 40616122

In [59]:
def parsing(data):
    processed = process_pipeline.process(data)
    content = [i for i in processed.split('\n') if not i.startswith('#')]
    return content
with open('output.conllu', 'w', encoding='utf-8') as f:
    f.write(str(parsing(text)))

In [9]:
cncn = morph.parse('автобрея')[0].tag
cnc = re.sub(' ', ',', str(cncn))
tatat = cnc.split(',')
itogog = ','.join(tatat[1:])
print(itogog)

impf,tran,pres


In [33]:
morph.parse('парными')

[Parse(word='парными', tag=OpencorporaTag('ADJF plur,ablt'), normal_form='парный', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'парными', 34, 25),)),
 Parse(word='парными', tag=OpencorporaTag('NOUN,inan,femn plur,ablt'), normal_form='парная', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'парными', 115, 11),)),
 Parse(word='парными', tag=OpencorporaTag('ADJF plur,ablt'), normal_form='парной', score=0.3333333333333333, methods_stack=((<DictionaryAnalyzer>, 'парными', 136, 25),))]

In [14]:
n3 = ('v', 'b', '', '')
'\t'.join(n3)

'v\tb\t\t'

In [29]:
gold_pipe = []

with open('processed (1).conllu', 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    print(lines[:10])
    for line in lines:
        if re.match(r'\d\d*\t', line):
            parts = line.split('\t')
            if parts[5] == '_':
                nPOS = ''
            else:
                nPOS_list = []
                for pair in parts[5].split('|'):
                    nPOS_list.append(pair.split('=')[1].lower())
                nPOS = ','.join(nPOS_list)
            g = (parts[1], parts[2], parts[3], nPOS)
            gold_pipe.append(g)
            
wrtitdown('golden_pipe.txt', gold_pipe)

['# newdoc', '# newpar', '# sent_id = 1', '# text = Цветки дикорастущих форм обыкновенной сирени мелкие, собраны парными пучками (по 3 - 5 цветков в пучке, отдельные сорта по 7 - 9 цветков).', '1\tЦветки\tЦветка\tNOUN\t_\tAnimacy=Inan|Case=Nom|Gender=Fem|Number=Plur\t8\tnsubj:pass\t_\t_', '2\tдикорастущих\tдикорастущий\tADJ\t_\tCase=Gen|Degree=Pos|Number=Plur\t3\tamod\t_\t_', '3\tформ\tформа\tNOUN\t_\tAnimacy=Inan|Case=Gen|Gender=Fem|Number=Plur\t1\tnmod\t_\t_', '4\tобыкновенной\tобыкновенный\tADJ\t_\tCase=Gen|Degree=Pos|Gender=Fem|Number=Sing\t5\tamod\t_\t_', '5\tсирени\tсирена\tNOUN\t_\tAnimacy=Inan|Case=Gen|Gender=Fem|Number=Sing\t3\tnmod\t_\t_', '6\tмелкие\tмелкий\tADJ\t_\tCase=Nom|Degree=Pos|Number=Plur\t1\tamod\t_\tSpaceAfter=No']


In [36]:
morph.parse('путча')

[Parse(word='путча', tag=OpencorporaTag('NOUN,inan,masc sing,gent'), normal_form='путч', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'путча', 79, 1),))]

In [39]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9]
a[1:-1]
a[-1]

9

In [56]:
a = '12345'

for c in range(0, len(a)):
    print(c)

0
1
2
3
4


In [62]:
fnames = ['GoldStandard.txt', 'pymorph_standard_nodouble.txt']

def getlines(fn):
    with open(fn, 'r', encoding='utf-8') as f:
        text = f.read()
        text = re.sub('ё', "е", text)
        text = re.sub("Ё", "е", text)
        lines = text.split('\n')[1:-1]
    return lines

GSr = getlines(fnames[0])
PMr = getlines(fnames[1])

GS = []

for line in GSr:
    if not re.search('\t\t\t', line):
        GS.append(line.lower())

i = 0
PM = []

for line in PMr:
    word = GS[i].split('\t')[0]
    word = re.sub('\.', '', word)
    w2 = line.split('\t')[0]
    if word == w2:
        PM.append(line)
        i += 1
        
PM = PM[:1000]
GS = GS[:1000]
        
with open('1000pm.txt', 'w', encoding='utf-8') as f:
    for line in PM:
        f.write(line)
        f.write('\n')
        
with open('1000gs.txt', 'w', encoding='utf-8') as f:
    for line in GS:
        f.write(line)
        f.write('\n')
        


cntdict = {}
        
for c in range(0, 1000):
    pmp = PM[c].split('\t')[2]
    gsp = GS[c].split('\t')[2]
    key = '%s:%s' % (gsp, pmp)
    if key not in cntdict:
        cntdict[key] = 1
    else:
        cntdict[key] += 1
        
total = 0
        
for key in sorted(cntdict, key=cntdict.get, reverse=True):
    mes = '%s\t:\t%d' % (key, cntdict[key])
    print(mes)
    total += cntdict[key]
    
print(total)

s:NOUN	:	356
pr:PREP	:	128
a:ADJF	:	127
v:VERB	:	86
conj:CONJ	:	58
adv:ADVB	:	40
spro:NPRO	:	32
apro:ADJF	:	31
v:INFN	:	29
adv:PRCL	:	21
v:PRTF	:	12
v:ADJF	:	11
apro:NPRO	:	10
v:PRTS	:	7
s:ADJS	:	5
adv:PRED	:	5
a:ADJS	:	4
adv:CONJ	:	4
s:UNKN	:	4
v:GRND	:	4
a:NOUN	:	3
anum:ADJF	:	3
adv:ADJS	:	2
v:NOUN	:	2
num:NUMR	:	2
spro:CONJ	:	2
adv:COMP	:	2
adv:NOUN	:	1
s:VERB	:	1
s:ADVB	:	1
anum:NOUN	:	1
spro:ADJF	:	1
spro:PRCL	:	1
s:ADJF	:	1
apro:ADJS	:	1
spro:ADVB	:	1
a:PRTF	:	1
1000


In [67]:
wr = []
rg = 0

for i in range(0, 1000):
    if GS[i].split('\t')[1].lower() == PM[i].split('\t')[1].lower():
        rg += 1
    else:
        wr.append((GS[i].split('\t')[1].lower(), PM[i].split('\t')[1].lower()))
        
rg

932

In [64]:
i = 0
s = (11, 10, 4, 4, 3, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1)
for c in s:
    i += c

i

45

In [104]:
tags = {}

with open('tagg.txt', 'r', encoding='utf-8') as f:
    plain = f.read()
    lines = plain.split('\n')
    for line in lines:
        if ':' in line:
            gstag, pmtag = line.split(':')
            tags[pmtag] = gstag

amb = []

with open('1000pm_ambig.txt', 'r', encoding='utf-8') as f:
    plain = f.read()
    lines = plain.split('\n')
    for line in lines:
        if not 'NUMB' in line:
            if not 'PNCT' in line:
                ln = line.split('\t')
                if len(amb) > 0:
                    word = amb[-1][0]
                    if word == ln[0]:
                        ln.append(0)
                    else:
                        ln.append(1)
                    amb.append(ln)
                else:
                    ln.append(1)
                    amb.append(ln)

amb = amb[:-1]
    
d = {}
b = 0
for line in amb:
    b += line[4]
    if b not in d:
        d[b] = [line[0], [line[3]]]
    else:
        d[b][1].append(line[3])

GSl = []       
        
for line in GS:        
    l = line.split('\t')
    GSl.append(l)
    
print(len(GSl))
    
lc = 0

nd = {}

print(d[12])

for key in sorted(d):
    word = d[key][0].lower()
    word = re.sub('ё', "е", word)
    worl = GSl[lc][0].lower()
    worl = re.sub('ё', "е", worl)
    worl = re.sub('\.', '', worl)
    if word == worl:
        gstags = GSl[lc][3].split(',')
        total = 0
        for gt in gstags:
            if gt in tags.values():
                total += 1
        acc = []
        if d[key][1][0] == '':
            fin = 1.0
        else:
            for var in d[key][1]:
                ts = var.split(',')
                curacc = 0
                for t in ts:
                    if t in tags:
                        nt = tags[t]
                        if nt in gstags:
                            curacc += 1
                if total > 0:
                    acc.append(round(curacc / total, 3))
                else:
                    acc.append(1.0)
            fin = max(acc)
        nd[key] = [word, fin]
        lc += 1
    
answer = 0
fail = []
    
for key in nd:
    if nd[key][1] < 1.0:
        answer += 1
        fail.append([key, nd[key]])
        
print(answer)
for fa in fail:
    print(fa)

1000
['в', ['']]
35
[7, ['собраны', 0.75]]
[39, ['морозостоек', 0.667]]
[108, ['полтриллиона', 0.667]]
[181, ['сел', 0.0]]
[189, ['готовы', 0.5]]
[204, ['году', 0.667]]
[221, ['все', 0.0]]
[272, ['виду', 0.667]]
[283, ['запущен', 0.8]]
[298, ['сделаны', 0.75]]
[331, ['редки', 0.5]]
[378, ['этого', 0.667]]
[417, ['чем-либо', 0.5]]
[445, ['состояния', 0.333]]
[460, ['все', 0.5]]
[499, ['нужна', 0.667]]
[515, ['газеты', 0.333]]
[518, ['агентства', 0.333]]
[561, ['борзых', 0.0]]
[567, ['гляди', 0.5]]
[588, ['этого', 0.667]]
[624, ['сотвори', 0.5]]
[629, ['люби', 0.5]]
[640, ['скорби', 0.5]]
[670, ['тартасов', 0.333]]
[678, ['таков', 0.667]]
[702, ['собой', 0.0]]
[729, ['стороны', 0.333]]
[804, ['принято', 0.8]]
[825, ['сказанным', 0.8]]
[830, ['совершенного', 0.6]]
[840, ['все', 0.5]]
[879, ['записи', 0.667]]
[880, ['решения', 0.333]]
[893, ['действия', 0.333]]


In [112]:
utags = {}
with open('utagg.txt', 'r', encoding='utf-8') as f:
    plain = f.read()
    lines = plain.split('\n')
    for line in lines:
        if ':' in line:
            gstag, pmtag = line.split(':')
            utags[pmtag] = gstag

In [136]:

with open('golden_pipe.txt', 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    lines = lines[1:1240]
    print(lines[-2:])
    UDr = []
    UD = []
    for line in lines:
        UDr.append(line.split('\t'))
    for line in UDr:
        if line[2] != 'PUNCT':
            if line[2] != "NUM":
                UD.append(line)
                
with open('1000udp.txt', 'w', encoding='utf-8') as f:
    for l in UD:
        f.write('\t'.join(l))
        f.write('\n')

len(UD)

['юридическим\tюридический\tADJ\tins,pos,neut,sing', 'лицом\tлицо\tNOUN\tinan,ins,neut,sing']


1015

In [143]:
nud = {}
uc = 0
ucntd = {}
lexc = 0

for item in UD:
    word = item[0].lower()
    word = re.sub('ё', "е", word)
    word = re.sub('\.', '', word)
    worl = GSl[uc][0].lower()
    worl = re.sub('ё', "е", worl)
    worl = re.sub('\.', '', worl)
    if '-' in worl:
        worl = worl.split('-')[0]
    if '-' in word:
        word = word.split('-')[0]
    if word == worl:
        gstags = GSl[uc][3].split(',')
        total = 0
        for gt in gstags:
            if gt in utags.values():
                total += 1
        acc = []
        if item[3] == '':
            fin = 1.0
        else:
            ts = item[3].split(',')
            curacc = 0
            for t in ts:
                if t in utags:
                    nt = utags[t]
                    if nt in gstags:
                        curacc += 1
            if total > 0:
                acc.append(round(curacc / total, 3))
            else:
                acc.append(1.0)
            fin = max(acc)
        nud[uc] = [word, fin]
        upos = item[2]
        gpos = GSl[uc][2]
        key = '%s:%s' % (upos, gpos)
        if key not in ucntd:
            ucntd[key] = 1
        else:
            ucntd[key] += 1
        ulex = item[1].lower()
        glex = GSl[uc][1].lower()
        if ulex == glex:
            lexc += 1
        uc += 1

print('lexeme', lexc)
        
answer = 0
fail = []
    
for key in nud:
    if nud[key][1] < 1.0:
        answer += 1
        fail.append([key, nud[key]])
        
print('gram', answer)
        
print('######')
        
totalu = 0
        
for key in sorted(ucntd, key=ucntd.get, reverse=True):
    mes = '%s\t:\t%d' % (key, ucntd[key])
    print(mes)
    totalu += ucntd[key]
    
print(totalu)

lexeme 937
gram 71
######
NOUN:s	:	335
VERB:v	:	136
ADJ:a	:	130
ADP:pr	:	128
ADV:adv	:	49
CCONJ:conj	:	46
PRON:spro	:	36
DET:apro	:	35
PROPN:s	:	31
PART:adv	:	17
AUX:v	:	10
SCONJ:conj	:	10
PRON:apro	:	6
ADJ:v	:	4
ADJ:adv	:	4
NOUN:a	:	3
ADJ:anum	:	3
AUX:adv	:	2
NUMP:num	:	2
PART:conj	:	2
PROPN:a	:	1
PRON:adv	:	1
ADV:spro	:	1
NUMP:anum	:	1
VERB:s	:	1
ADJ:s	:	1
VERB:adv	:	1
NOUN:v	:	1
ADJ:apro	:	1
SCONJ:adv	:	1
VERB:a	:	1
1000
